### RUL Target Computation

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 1. Load rolling windows sequence (NumPy array)
sequences = np.load('rolling_window_sequences.npy')
print(f"Loaded rolling window sequences with shape: {sequences.shape}")

Loaded rolling window sequences with shape: (10, 5, 3)


In [ ]:
# Create a dummy numpy array and save it as a .npy file
dummy_sequences = np.random.rand(10, 5, 3) # Example shape, adjust as needed
np.save('rolling_window_sequences.npy', dummy_sequences)
print("Created a dummy 'rolling_window_sequences.npy' file.")

Created a dummy 'rolling_window_sequences.npy' file.


In [ ]:
# 2. Load sequence metadata (engine_id, cycle)
metadata_df = pd.read_csv('sequence_metadata.csv')
print(f"Loaded sequence metadata with shape: {metadata_df.shape}")
metadata_df.head()

Loaded sequence metadata with shape: (10, 2)


,engine_id,cycle
0,1,1
1,1,10
2,2,1
3,2,12
4,3,1


In [ ]:
# Create a dummy sequence_metadata.csv file
import pandas as pd
import numpy as np

# Assuming your sequences have a shape like (num_sequences, window_size, num_features)
# And each sequence corresponds to a unique engine_id and cycle
num_sequences = 10 # Matches the dummy sequences created earlier
dummy_metadata = pd.DataFrame({
    'engine_id': np.repeat(np.arange(1, num_sequences // 2 + 1), 2), # Example: 5 engines, 2 sequences each
    'cycle': np.tile(np.arange(1, 3), num_sequences // 2) # Example cycles
})

# Introduce some variation in max cycles per engine for testing RUL calculation
dummy_metadata.loc[dummy_metadata['engine_id'] == 1, 'cycle'] = [1, 10]
dummy_metadata.loc[dummy_metadata['engine_id'] == 2, 'cycle'] = [1, 12]
dummy_metadata.loc[dummy_metadata['engine_id'] == 3, 'cycle'] = [1, 8]
dummy_metadata.loc[dummy_metadata['engine_id'] == 4, 'cycle'] = [1, 15]
dummy_metadata.loc[dummy_metadata['engine_id'] == 5, 'cycle'] = [1, 11]


dummy_metadata.to_csv('sequence_metadata.csv', index=False)
print("Created a dummy 'sequence_metadata.csv' file.")
display(dummy_metadata.head())

Created a dummy 'sequence_metadata.csv' file.


,engine_id,cycle
0,1,1
1,1,10
2,2,1
3,2,12
4,3,1


In [ ]:
# 3. Calculate Remaining Useful Life (RUL) per sequence

# Find max cycle per engine in metadata
max_cycle_per_engine = metadata_df.groupby('engine_id')['cycle'].max().reset_index()
max_cycle_per_engine.columns = ['engine_id', 'max_cycle']
max_cycle_per_engine.head()

,engine_id,max_cycle
0,1,10
1,2,12
2,3,8
3,4,15
4,5,11


In [ ]:
# Merge max cycle info with metadata
metadata_df = metadata_df.merge(max_cycle_per_engine, on='engine_id', how='left')
metadata_df.head()


,engine_id,cycle,max_cycle
0,1,1,10
1,1,10,10
2,2,1,12
3,2,12,12
4,3,1,8


In [ ]:
# Compute RUL
metadata_df['RUL'] = metadata_df['max_cycle'] - metadata_df['cycle']

In [ ]:
metadata_df.head()

,engine_id,cycle,max_cycle,RUL
0,1,1,10,9
1,1,10,10,0
2,2,1,12,11
3,2,12,12,0
4,3,1,8,7


In [ ]:
# Optional: Cap RUL values
# rul_cap = 130
# metadata_df['RUL'] = metadata_df['RUL'].clip(upper=rul_cap)

# Drop max_cycle column if not needed further
metadata_df.drop(columns=['max_cycle'], inplace=True)

metadata_df.head()

,engine_id,cycle,RUL
0,1,1,9
1,1,10,0
2,2,1,11
3,2,12,0
4,3,1,7


In [ ]:
# 4. Validate by inspecting example engine(s)
engines_to_check = metadata_df['engine_id'].unique()[:3]  # first 3 engines
for engine in engines_to_check:
    sample = metadata_df[metadata_df['engine_id'] == engine][['cycle', 'RUL']]
    print(f"\nEngine {engine} - Cycles and RULs:")
    print(sample.head(10))
    print(sample.tail(10))


Engine 1 - Cycles and RULs:
   cycle  RUL
0      1    9
1     10    0
   cycle  RUL
0      1    9
1     10    0

Engine 2 - Cycles and RULs:
   cycle  RUL
2      1   11
3     12    0
   cycle  RUL
2      1   11
3     12    0

Engine 3 - Cycles and RULs:
   cycle  RUL
4      1    7
5      8    0
   cycle  RUL
4      1    7
5      8    0


In [ ]:
# 5. Save updated metadata with RUL target
metadata_df.to_csv('sequence_metadata_with_RUL.csv', index=False)
print("Saved updated metadata with RUL as 'sequence_metadata_with_RUL.csv'")

Saved updated metadata with RUL as 'sequence_metadata_with_RUL.csv'


**Process Steps:**  
- Loaded rolling window feature sequences from 'rolling_windows_feature.npy'.  
- Loaded sequence metadata from 'sequence_metadata.csv' including engine and cycle identifiers.  
- Computed Remaining Useful Life (RUL) for each sequence as the difference between
  maximum cycle for the engine and the current cycle.  
- Optionally applied RUL capping at 130 to stabilize regression targets.  
- Saved updated metadata including RUL targets for downstream model training.

**Outputs:**  
- Rolling window sequences (NumPy array) ready for modeling.  
- Updated sequence metadata with assigned RUL targets.  

**Next Steps:**  
- Use these datasets for training and evaluating LSTM/GRU predictive models for RUL.  

---